# Accounting for seasonality

## Examining the SARIMA(p,d,q)(P,D,Q)m model

#### The SARIMA(p,d,q)(P,D,Q)m model expands on the ARIMA(p,d,q) model from the previous chapter by adding seasonal parameters. You’ll notice four new parameters in the model: P, D, Q, and m. The first three have the same meaning as in the ARIMA(p,d,q) model, but they are their seasonal counterparts. To understand the meaning of these parameters and how they affect the final model, we must first define m.

#### The parameter m stands for the frequency. In the context of a time series, the frequency is defined as the number of observations per cycle. The length of the cycle will depend on the dataset. For data that was recorded every year, quarter, month, or week, the length of a cycle is considered to be 1 year. If the data was recorded annually, m = 1 since there is only one observation per year. If the data was recorded quarterly, m = 4 since there are four quarters in a year, and therefore four observations per year. Of course, if the data was recorded monthly, m = 12. Finally, for weekly data, m = 52. Table 8.1 indicates the appropriate value of m depending on the frequency at which the data was collected.



#### When data is collected on a daily or sub-daily basis, there are multiple ways of interpreting the frequency. For example, daily data can have a weekly seasonality. In that case, the frequency is m = 7 because there would be seven observations in a full cycle of 1 week. It could also have a yearly seasonality, meaning that m = 365. Thus, you can see that daily and sub-daily data can have a different cycle length, and therefore a different frequency m. Table 8.2 provides the appropriate value of m depending on the seasonal cycle for daily and sub-daily data.

#### Note that a SARIMA(p,d,q)(0,0,0)m model is equivalent to an ARIMA(p,d,q) model.

#### Let’s consider an example where m = 12. If P = 2, this means that we are including two past values of the series at a lag that is a multiple of m. Therefore, we’ll include the values at yt–12 and yt–24.

#### Similarly, if D = 1, this means that a seasonal difference makes the series stationary. In this case, a seasonal difference would be expressed as equation 8.1.

#### y't = yt– yt–12

## Identifying seasonal patterns in a time series

## Time series decomposition

- Time series decomposition is a statistical task that separates the time series into its three main components: a trend component, a seasonal component, and the residuals.

#### The trend component represents the long-term change in the time series. This component is responsible for time series that increase or decrease over time. The seasonal component is the periodic pattern in the time series. It represents repeated fluctuations that occur over a fixed period of time. Finally, the residuals, or the noise, express any irregularity that cannot be explained by the trend or the seasonal component.